## PS :- Build a System to identify unrealiable news article

train.csv: A full training dataset with the following attributes:

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

1: unreliable

0: reliable

test.csv: A testing training dataset with all the same attributes at train.csv without the label.

submit.csv: A sample submission that you can

In [68]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [69]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [70]:
## here is several pagkages to import

import pandas as pd
import numpy as np

In [71]:
import os
for dirname, _, filenames in os.walk('D:\Capstone_project1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:\Capstone_project1\submit.csv
D:\Capstone_project1\test.csv
D:\Capstone_project1\train.csv


In [72]:
from nltk.corpus import stopwords

In [73]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [75]:
## Data Preprocessing

In [76]:
news = pd.read_csv(r"D:\Capstone_project1\train.csv")

In [77]:
news.shape

(20800, 5)

In [78]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [79]:
## Lets check the nulls values in that dataset

In [80]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [81]:
## from above there are no missing values in id and label
## but in other like title,author and text we can replace it with appropriate values

In [82]:
#news.title.value_counts()

In [83]:
# replacing null values with empty string
news = news.fillna('')

In [84]:
## Lets check handled null values
news.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [85]:
 ## Lets combining column author and title

In [86]:
news.dtypes

id         int64
title     object
author    object
text      object
label      int64
dtype: object

In [87]:
## For our prediction we are going to include title and author
## so basically we combine both column together so this is the data we are going to use th eprediction
## we are not using text column because this text column are too much large and it takes lot of time for processing

In [88]:
## lets merge the title and author together
news['content'] = news['author']+' '+ news['title']

In [108]:
news1 =news.drop(columns=['title','author'])

In [109]:
news1.head()

,id,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [89]:
 ## here we created column name content concantes the strings

In [110]:
news['content']      ## here we can see that author name first and next title name 

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
5        Daniel Nussbaum Jackie Mason: Hollywood Would ...
6         Life: Life Of Luxury: Elton John’s 6 Favorite...
7        Alissa J. Rubin Benoît Hamon Wins French Socia...
8         Excerpts From a Draft Script for Donald Trump...
9        Megan Twohey and Scott Shane A Back-Channel Pl...
10       Aaron Klein Obama’s Organizing for Action Part...
11       Chris Tomlinson BBC Comedy Sketch "Real Housew...
12       Amando Flavio Russian Researchers Discover Sec...
13       Jason Ditz US Officials See No Link Between Tr...
14       AnotherAnnie Re: Yes, There Are Paid Governmen...
15       Jack Williams In Major League Soccer, Argentin...
16       Michael Corkery and Stacy Cowley Wells Fargo C.

In [91]:
## so that we can use this data for prediction

In [114]:
x = news1.drop(columns='label', axis=1)
y = news1['label']

In [115]:
#print(x)
x.head()

,id,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


In [116]:
print(y)

0        1
1        0
2        1
3        1
4        1
5        0
6        1
7        0
8        0
9        0
10       0
11       0
12       1
13       1
14       1
15       0
16       0
17       1
18       1
19       0
20       1
21       0
22       0
23       1
24       0
25       1
26       0
27       1
28       0
29       0
30       0
31       1
32       0
33       0
34       0
35       0
36       1
37       1
38       0
39       0
40       0
41       1
42       0
43       1
44       0
45       0
46       1
47       1
48       0
49       0
50       1
51       1
52       0
53       1
54       1
55       1
56       1
57       1
58       1
59       0
60       0
61       1
62       0
63       1
64       0
65       0
66       1
67       1
68       0
69       1
70       1
71       1
72       0
73       0
74       0
75       0
76       1
77       1
78       0
79       0
80       0
81       0
82       1
83       1
84       0
85       0
86       1
87       0
88       1
89       0
90       0

In [ ]:
## We need to do is stemming procedure

In [ ]:
## Stemming :- it is a process of reducing a word to each root word
# e.g actor , actress , acting ---> Act

In [37]:
## we reduce this words as muchh as posiible to get better performance on our model
## so once we do this procedure of reducing the words to its accurate root word we use function of vectorizing
## so in that vectorizing process we will convert these word to their you know respective feature vectors
# feature vectors basically is numerical data so once we convert textt to numerical data we can feed it to our ML model

In [41]:
## we use this portersTEMMER for further process
from nltk.stem.porter import PorterStemmer
port_stem = PorterStemmer()

def stemming(content):    ## creating function named stemming and in bracket we need to use input value to give it
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()  ##converting all those alphabates in lower case
    stemmed_content = stemmed_content.split()  ## split data set into list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] ## removing all stopwords and choosing those who are not stopwords
    stemmed_content = ' '.join(stemmed_content)  # join all the words
    return stemmed_content

In [42]:
## we need to apply this function on content column
news['content'] = news['content'].apply(stemming)

In [45]:
news['content']  ## it shows there is no upper case letters and it mapped to its root words

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
5        daniel nussbaum jacki mason hollywood would lo...
6        life life luxuri elton john favorit shark pict...
7        alissa j rubin beno hamon win french socialist...
8        excerpt draft script donald trump q ampa black...
9        megan twohey scott shane back channel plan ukr...
10       aaron klein obama organ action partner soro li...
11       chri tomlinson bbc comedi sketch real housew i...
12       amando flavio russian research discov secret n...
13              jason ditz us offici see link trump russia
14       anotheranni ye paid govern troll social media ...
15       jack william major leagu soccer argentin find ...
16       michael corkeri staci cowley well fargo chief .

In [46]:
## Separting the data and label
x= news['content'].values
y = news['label'].values

In [100]:
news.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [48]:
print(x)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [ ]:
## 1 = fake news
# 0 - real news

In [49]:
y.shape

(20800,)

In [ ]:
## so value is still in textuare form so computers cant understand text so we have need to convert all this value into meaningful
# numbers that computers can understand so for this we use vectorizing function

In [50]:
# Lets converting texual data to numerical data
vectorizer = TfidfVectorizer() #Tf stands for term frequency & idf is inverse document freq so it basically counts no of times particular word is repeating in document
vectorizer.fit(x) #fitting the vectorizer values and x is nothing but content column

X = vectorizer.transform(x)  # converting text to no that system can understand

In [52]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [ ]:
## now that numerical data understand system so we haVE to do sampling i.e split it into train anad test

In [53]:
from sklearn.model_selection  import train_test_split

In [54]:
X_train ,X_test , y_train , y_test = train_test_split(X , y ,test_size = .2)  ## 20% data in test

In [ ]:
## Lets importing logistic Regression model function

In [55]:
model = LogisticRegression()

In [57]:
model.fit(X_train , y_train)

LogisticRegression()

In [ ]:
## Lets evaluate further

In [ ]:
# lets do prediction 1st on training data

In [59]:
X_train_prediction = model.predict(X_train)
accuracy_train = accuracy_score(X_train_prediction , y_train)
accuracy_train

0.9868389423076923

In [ ]:
# we trained model on training data ad prediction using test data

In [ ]:
## lets find accuracy score on test data so it tells us that how good our model is performing

In [60]:
X_test_prediction = model.predict(X_test)
accuracy_test = accuracy_score(X_test_prediction , y_test)
accuracy_test

0.9754807692307692

In [ ]:
## it verymuch equal to our train data  so that means model has not overtrained with traing data it is performing very well 
# so we sucessfully trained our model

In [ ]:
## now lets predict the model that news is unreliable or reliable

In [63]:
new = X_test[0]    # 0 means first row in our expressed column

prediction = model.predict(new)
print(prediction)

if (prediction[0]== 0):
    print('News is Reliable')
else:
    print('News is Unreiable')

[0]
News is Reliable


In [67]:
y_test[6]

1